In [1]:
from __future__ import division
import random
import pprint
import sys
import time
import numpy as np
from optparse import OptionParser
import pickle

from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Input
from keras.models import Model
from keras_frcnn import config, data_generators
from keras_frcnn import losses as losses
from keras_frcnn import resnet as nn
import keras_frcnn.roi_helpers as roi_helpers
from keras.utils import generic_utils
from keras_frcnn.simple_parser import get_data

sys.setrecursionlimit(40000)

C = config.Config()


all_imgs, classes_count, class_mapping = get_data("apple_images/fout_test2.csv")

if 'bg' not in classes_count:
	classes_count['bg'] = 0
	class_mapping['bg'] = len(class_mapping)

C.class_mapping = class_mapping

inv_map = {v: k for k, v in class_mapping.items()}

print('Training images per class:')
pprint.pprint(classes_count)
print('Num classes (including bg) = {}'.format(len(classes_count)))

with open("config.pickle", 'rb') as f_in:
	C = pickle.load(f_in)

C.base_net_weights = "model_frcnn.hdf5"
    
num_imgs = len(all_imgs)

train_imgs = [s for s in all_imgs if s['imageset'] == 'trainval']
val_imgs = [s for s in all_imgs if s['imageset'] == 'test']

print('Num train samples {}'.format(len(train_imgs)))
print('Num val samples {}'.format(len(val_imgs)))


data_gen_train = data_generators.get_anchor_gt(train_imgs, classes_count, C, K.image_dim_ordering(), mode='train')
data_gen_val = data_generators.get_anchor_gt(val_imgs, classes_count, C, K.image_dim_ordering(), mode='val')

if K.image_dim_ordering() == 'th':
	input_shape_img = (3, None, None)
else:
	input_shape_img = (None, None, 3)

img_input = Input(shape=input_shape_img)
roi_input = Input(shape=(C.num_rois, 4))

# define the base network (resnet here, can be VGG, Inception, etc)
shared_layers = nn.nn_base(img_input, trainable=True)

# define the RPN, built on the base layers
num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
rpn = nn.rpn(shared_layers, num_anchors)

classifier = nn.classifier(shared_layers, roi_input, C.num_rois, nb_classes=len(classes_count), trainable=True)

model_rpn = Model(img_input, rpn[:2])
model_classifier = Model([img_input, roi_input], classifier)

# this is a model that holds both the RPN and the classifier, used to load/save weights for the models
model_all = Model([img_input, roi_input], rpn[:2] + classifier)

try:
	print('loading weights from {}'.format(C.base_net_weights))
	model_rpn.load_weights(C.base_net_weights, by_name=True)
	model_classifier.load_weights(C.base_net_weights, by_name=True)
except e:
	print(e)
finally:
    print("loaded!")
    
optimizer = Adam(lr=1e-4)
optimizer_classifier = Adam(lr=1e-4)
model_rpn.compile(optimizer=optimizer, loss=[losses.rpn_loss_cls(num_anchors), losses.rpn_loss_regr(num_anchors)])
model_classifier.compile(optimizer=optimizer_classifier, loss=[losses.class_loss_cls, losses.class_loss_regr(len(classes_count)-1)], metrics={'dense_class_{}'.format(len(classes_count)): 'accuracy'})
model_all.compile(optimizer='sgd', loss='mae')

predictions=model_all.predict_on_batch(all_imgs)

print('Training complete, exiting.')


Using TensorFlow backend.


Parsing annotation files
Training images per class:
{' apple': 10, 'bg': 0}
Num classes (including bg) = 2
Num train samples 6
Num val samples 0
loading weights from model_frcnn.hdf5
loaded!


ValueError: Error when checking model : you are passing a list as input to your model, but the model expects a list of 2 Numpy arrays instead. The list you passed was: [{'imageset': 'trainval', 'bboxes': [{'y2': 769, 'x2': 490, 'x1': 445, 'y1': 732, 'class': ' apple'}], 'filepath': 'apple_images/01_Enrico_Carbognani/conv/P_20170625_184035.jpg', 'width': 720, 'height

In [2]:
!ls apple_images

01_Enrico_Carbognani	09_Sofia_Asteria_Piffer  fout.csv
02_Paolo_Iachemet	10_Surabhi_Mundana	 fout_test.csv
03_Sachin_Konan		11.1_Elliot		 fout_train.csv
04_Andrea_Zanin		11.2_Balgeet_Tjinder	 join_csvs.py
05_Aurnav_Nardelli	12_Mihir_Patel		 jsonsloth_to_csv.py
06_Gregorio_Giovanazzi	13_Matteo_Albi		 pipeline.sh
07_Marco_Zamboni	14_Nunzio_Fallico	 test_imgs
08_Martina_Parisi	annotation.json


In [29]:
np.random.randint(0, 255, 30)

array([173,  58, 189, 238, 131,   3, 162, 178, 199, 126, 165, 201,  37,
        83,  95, 250, 136,  66,  24,  83, 154,  34, 213, 102, 253, 190,
        40,  37, 116, 207])